# Q# Interop with OpenQASM

The modern QDK provides interoperability with OpenQASM 3 programs built upon the core Q# compiler infrastructure.

This core enables integration and local resource estimation without relying on external tools. Users are able to estimate resources for their OpenQASM programs locally (see the [resource estimation with Qiskit sample notebook](../../estimation/estimation-openqasm.ipynb)), leveraging the Q# compiler's capabilities for analysis, transformation, code generation, and simulation. This also enables the generation of QIR from OpenQASM progams leveraging the [modern QDKs advanced code generation capabilities](https://devblogs.microsoft.com/qsharp/integrated-hybrid-support-in-the-azure-quantum-development-kit/).

This includes support for classical instructions available in OpenQASM such as for loops, if statements, switch statements, while loops, binary expresssions, and more.

### Run OpenQASM 3 Code in interactive session
Interactive sessions have different semantics from program execution. We no longer have inferred output and input. Instead we treat qasm lines as code fragments and interpret them one at a time (though they are all compiled together). Due to scoping rules in OpenQASM, all code used in the the program must be defined in the snippet and can't use compilation state from other cells or calls.



Import the Q# module.

This enables the `%%openqasm` (and `%%qsharp`) magic and initializes a interpreter singleton.

In [1]:
import qsharp
qsharp.init(target_profile=qsharp.TargetProfile.Base)

Q# initialized with configuration: {'targetProfile': 'base', 'languageFeatures': None, 'manifest': None}

With the runtime initialized we can now run OpenQASM programs in their own cells with the `%%qasm` magic. Running this cell will simulate the program using the default simulator and its output will be displayed.

In [ ]:
%%openqasm --shots 1024
include "stdgates.inc";
qubit[2] q;
h q[0];
cx q[0], q[1];
bit[2] c;
c[0] = measure q[0];
c[1] = measure q[1];

We can leverage the Jupyter `_` variable to access the run output from the previous cell  if we want to work with the output.

In [ ]:
last_output = _ # type: ignore
print(last_output)

We can add an optional `name` parameter to compile the program into a callable operation in the interactive session.

In [ ]:
%%openqasm --name bell
include "stdgates.inc";
qubit[2] q;
h q[0];
cx q[0], q[1];
qit[2] c;
c[0] = measure q[0];
c[1] = measure q[1];

With the OpenQASM program loaded into a callable name `bell`, we can now import it via the QDK's Python bindings:

In [ ]:
from qsharp.code import bell
bell()

Additionally, since it is defined in the session, we can run it directly from a Q# cell:

In [ ]:
%%qsharp
c

This also unlocks all of the other `qsharp` package functionality. Like noisy simulation:

In [ ]:
from qsharp_widgets import Histogram

Histogram(qsharp.run("bell()", shots=1000, noise=qsharp.DepolarizingNoise(0.01)))

### Circuit Rendering (textual)

In [ ]:
qsharp.circuit(qsharp.code.bell)

### Circuit Rendering (widget)

In [ ]:
from qsharp_widgets import Circuit
Circuit(qsharp.circuit(qsharp.code.bell))

### Compilation


In [ ]:
print(qsharp.compile(bell))

#### QIR generation semantic errors

When targetting harware by compiling to QIR there are additional restrictions which may cause compilation errors. Most common scenarios:
- Trying to generate QIR when the profile is set to `Unrestricted`. `Unrestricted` is only valid for simulation. Either `TargetProfile.Base` or `TargetProfile.Adaptive_RI` must be used.
- Not all bits in classical registers have been assigned to. Usually because there were no measurements, or extra registers were declared.



Example, generating QIR with `Unrestricted`

In [ ]:
%%openqasm
include "stdgates.inc";
qubit[3] q;
h q[1];
cx q[1], q[2];
cx q[0], q[1];
h q[0];
bit[2] c;
c[0] = measure q[0];
c[1] = measure q[1];
if (c[0] == 1) z q[2];
if (c[1] == 1) x q[2];


To avoid this issue, set the `target_profile` argument either in the `QSharpBackend` creation or in the `backend.qir` call.

When generating `QIR`, all output registers must be read into before generating QIR. Failure to do so results in a `QSharpError`.

In this next example, we declare two output bits, but only measure into one. This causes an error because result values can only be a side effect of measurement, and cannot be used like classical variables when compiling for hardware.

In [ ]:
circuit = QuantumCircuit(2, 2)
circuit.x(0)
circuit.measure(0, 1)
backend = QSharpBackend(target_profile=TargetProfile.Base)
try:
    print(backend.qir(circuit))
except QSharpError as ex:
    print(ex)